In [1]:
def check_tuple(var, typ):
    if type(var)!=tuple and type(var)!=typ:
        return False 
    elif type(var)==tuple:
        if any(type(s)!=typ for s in var):
            return False
    return True

In [158]:
def diff_matrix_1d(grid_size, difference_order=4, periodic_boundary=False):
    """
    Return a differentiation matrices associated with derivative operators up to 2nd order on the line
    The derivative is calculated with respect to the equispaced grid
    The differentiation matrices are sparsed.
    
    Parameters 
    ----------
    grid_size : int nx
    difference_order : int default=4
        Controls order of the approximation of finite difference, sets the size of the stencil
        The endpoint derivatives are calculated with larger stencil for better accuracy
    peryodic_boundary: boolean, default=False
        Sets whether periodic boundary conditions are imposed
    
    Returns
    ----------
    List of differentiation matrices of size nx X nx of rising order including identity. 
    I.e. [1, dx, d2x]
    
    References
    ---------------
    Uses sympy.calculus.finite_diff.finite_diff_weights
    """
    #----------------Check input------------------ 
    if any(type(inp)!=int for inp in (grid_size, difference_order)):
        raise ValueError("grid_size and difference_order must be integers")
    if type(periodic_boundary)!=bool:
        raise ValueError("periodic_boundary must be boolean")

    if difference_order+2 > grid_size:
        raise ValueError("Stencil is larger then the whole grid")
    
    #-------------Body------------------    
    
    h0=int(difference_order/2) #This is a number of neighbours
    
    #calculate a set of derivatives with various offset and single out the central one
    #the endpoint derivatives are calculated with larger stencil for better accuracy
    weights_all=[]
    for offset in range(-h0,h0+1,1):
        if offset==0:
            stencil=[S(i) for i in range(-h0,h0+1)]
            weights_all.append(np.array(finite_diff_weights(2, stencil, offset),dtype=float)[:,-1])
        if offset<0:
            stencil=[S(i) for i in range(2*h0+2)]
            weights_all.append(np.array(finite_diff_weights(2, stencil, h0+offset),dtype=float)[:,-1])
        if offset>0:
            stencil=[S(i) for i in range(2*h0+2)]
            weights_all.append(np.array(finite_diff_weights(2, stencil, h0+1+offset),dtype=float)[:,-1])
    
    weights_center=weights_all[h0]
    
    
    #write a diff matrix of order 0 (identity) to the list
    diff_mats=[scipy.sparse.identity(grid_size)]
    
    #create the matrices for 1st and 2nd derivatives
    #non-periodic boundary
    if periodic_boundary==False:
        for deriv in range(1,3):
            #away from the boundaries the central derivatives fill the diagonal
            weights=weights_center[deriv]
            diags=np.broadcast_to(weights,(grid_size,2*h0+1)).transpose()
            in_mat=scipy.sparse.dia_matrix((diags,range(2*h0+1)),shape=(grid_size-(2*h0),grid_size)) 


            #near the bottom (beginnig of the grid) use one-sided derivatives in the left part of matrix, fill with zeros to the right
            bottom_mat_right=scipy.sparse.dia_matrix((h0,grid_size-(2*h0+2))) 
            bottom_mat_left=np.zeros((h0,2*h0+2))
            for dist in range(h0):
                bottom_mat_left[dist] = weights_all[dist][deriv]
            bottom_mat=scipy.sparse.hstack([bottom_mat_left,bottom_mat_right])

            #same near top, fill with zeros to the left
            top_mat_left=scipy.sparse.dia_matrix((h0,grid_size-(2*h0+2))) 
            top_mat_right=np.zeros((h0,2*h0+2))
            for dist in range(-h0,0):
                top_mat_right[dist] = weights_all[2*h0+1+dist][deriv]
            top_mat=scipy.sparse.hstack([top_mat_left,top_mat_right])

            full_mat=scipy.sparse.vstack([bottom_mat,in_mat,top_mat])

            diff_mats.append(full_mat)
    
    #Periodic boundary        
    if periodic_boundary==True:
        for deriv in range(1,3):
            #the central derivatives fill the diagonal plus show up in the corners
            weights=weights_center[deriv]
            diags=np.broadcast_to(weights,(grid_size,2*h0+1)).transpose()
            diags=np.vstack((diags[h0+1:],diags,diags[:h0]))
            offsets=list(range(-grid_size+1,h0+1-grid_size))+list(range(-h0,h0+1))+list(range(-h0+grid_size,grid_size))
            full_mat=scipy.sparse.dia_matrix((diags,offsets),shape=(grid_size,grid_size)) 

            diff_mats.append(full_mat)
        
    return diff_mats

In [164]:
dw=diff_matrix_1d(8,difference_order=6,periodic_boundary=False)

In [167]:
dw[1].toarray()

array([[ -2.59285714e+00,   7.00000000e+00,  -1.05000000e+01,
          1.16666667e+01,  -8.75000000e+00,   4.20000000e+00,
         -1.16666667e+00,   1.42857143e-01],
       [ -1.42857143e-01,  -1.45000000e+00,   3.00000000e+00,
         -2.50000000e+00,   1.66666667e+00,  -7.50000000e-01,
          2.00000000e-01,  -2.38095238e-02],
       [  2.38095238e-02,  -3.33333333e-01,  -7.83333333e-01,
          1.66666667e+00,  -8.33333333e-01,   3.33333333e-01,
         -8.33333333e-02,   9.52380952e-03],
       [ -1.66666667e-02,   1.50000000e-01,  -7.50000000e-01,
          0.00000000e+00,   7.50000000e-01,  -1.50000000e-01,
          1.66666667e-02,   0.00000000e+00],
       [  0.00000000e+00,  -1.66666667e-02,   1.50000000e-01,
         -7.50000000e-01,   0.00000000e+00,   7.50000000e-01,
         -1.50000000e-01,   1.66666667e-02],
       [ -9.52380952e-03,   8.33333333e-02,  -3.33333333e-01,
          8.33333333e-01,  -1.66666667e+00,   7.83333333e-01,
          3.33333333e-01,  -2.3

In [17]:
def diff_matrix(grid_size, difference_order=4, periodic_boundary=False):
    """
    Return a differentiation matrices associated with derivative operators up to 2nd order.
    The derivative is calculated with respect to the equispaced multidimensional grid
    The differentiation matrices are sparsed, obtained by Kronecker product. 
    
    Parameters 
    ----------
    ! All tuples must be the same lenght equal to the number of grid dimensions.
    
    grid_size : tuple of ints (nx,ny,...)
    difference_order : tuple of ints (int,int,...), default=(5,5)
        Controls order of the approximation of finite difference for every axis.
    periodic_boundary: tuple of booleans, default=(False,False)
        Sets whether periodic boundary conditions are imposed in given direction
    
    Returns
    ----------
    List of differentiation matrices of size (nx*ny*...)x(nx*ny*...) of rising order including identity. 
    I.e. in 2 dimensions
??    (1, dx, dy, d2x, d2y, dxdy)
    
    References
    ---------------
    Uses sympy.calculus.finite_diff.finite_diff_weights
    and scipy.sparse.kron
    """
    #-----------Check input--------------
    #-----------------------------------
    def to_tuple(var):
        if type(var)!=tuple:
            return tuple([var]*len(grid_size))
        else:
            return var
            
    if not check_tuple(difference_order,int):
        raise ValueError("difference_order must be integer or tuple of integers")
    if not check_tuple(periodic_boundary,bool):
        raise ValueError("periodic_boundary must be integer or tuple of integers")
 
    (difference_order, periodic_boundary)=map(to_tuple, (difference_order, periodic_boundary))
    
    dims=tuple(len(inp) for inp in [grid_size,stencil_size,  periodic_boundary])
    if any(s1!=s2 for s1, s2 in zip(dims,dims[1:])):
        raise ValueError("Dimensions of grid, order and periodicity flags are not equal.")

    
    #-------Body-----------------------
    #-----------------------------------
    
    from sympy.calculus.finite_diff import finite_diff_weights
    from sympy import S
    
    h0=int(difference_order/2) #This is a number of neighbours
    stencil=[S(i) for i in range(-h0,h0+1,1)]
    res = finite_diff_weights(2, stencil, -3)
    res[0][-1]
    
    return res

In [16]:
diff_matrix((3,4))

NameError: name 'diff_matrix' is not defined

In [64]:
stencil=[S(i) for i in range(-2,3,1)]
stencil

[-2, -1, 0, 1, 2]

In [65]:
h0=3
stencil=[S(i) for i in range(-h0,h0+1,1)]
res = finite_diff_weights(2, stencil, -3)
res[0][-1]

[1, 0, 0, 0, 0, 0, 0]

In [5]:
from sympy.calculus.finite_diff import finite_diff_weights
from sympy import S
import numpy as np

In [6]:
import scipy.sparse